In [1]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS, utils
import astropy.coordinates as coo
import astroalign as aa
from astropy.nddata import Cutout2D
from reproject import reproject_interp

import subprocess
import os
import sys
sys.path.append("../../Resolved_Polarization_v3")
sys.path.append("../../analysis_v3")
from resolved_pol import ResolvedPol

In [2]:
#Load the image with the worse seeing from W0204, as everything is being matched to it. 
fors2_im = fits.open("../../analysis_v3/bkg/science_reduced_img.W0204-0506.60209.R_SPECIAL.chip1.3.bkg.fits")
emask = fits.getdata("../../analysis_v3/masks/science_reduced_img.W0204-0506.60209.R_SPECIAL.chip1.3.emask.fits",dtype=bool)

In [3]:
#Load the F160W and F555W images which are already aligned to each other.
f555w = fits.open("id1j03010_drc_sci.ali.fits")
f160w = fits.open("id1j03020_drz.fits")

In [4]:
#It is too difficult to do this automatically, so we'll just use a few stars we have selected by hand. The centroids are eye-balled, so this can certainly be improved.
data = np.loadtxt("stars.txt")
f160w_pix = list()
fors2_pix = list()
for i in range(len(data)):
    f160w_pix.append((data[i,0]-1, data[i,1]-1))
    fors2_pix.append((data[i,2]-1, data[i,3]-1))
f160w_pix = np.array(f160w_pix)
fors2_pix = np.array(fors2_pix)

In [5]:
#Now, get the WCS coordinates of the stars using the F160W image WCS. 
wcs_f160w = WCS(f160w['SCI'].header)
f160w_sky = wcs_f160w.pixel_to_world(f160w_pix[:,0], f160w_pix[:,1])
print(f160w_sky)

<SkyCoord (ICRS): (ra, dec) in deg
    [(31.17419654, -5.11344987), (31.18740103, -5.10076933),
     (31.19265921, -5.11263911)]>


In [6]:
#Now, let's determine the WCS of the FORS2 image. 
fors2_wcs = utils.fit_wcs_from_points((fors2_pix[:,0], fors2_pix[:,1]), f160w_sky)

In [7]:
#The thing we know, however, is that the original image has north pointing directly up, since we know the WCS of the unprocessed image. Let's use that rotation matrix instead, we just need to shift the center. 
raw_fors2_header = fits.getheader("../../proc/W0204-0506/60209/IMAGE/CHIP1/R_SPECIAL/FORS2.2023-09-22T07:18:53.988.fits")


In [8]:
#Update the header
fors2_wcs_header = fors2_wcs.to_header()
for i in range(2):
    for j in range(2):
        fors2_wcs_header['PC{}_{}'.format(i+1,j+1)] = raw_fors2_header['CD{}_{}'.format(i+1,j+1)]
fors2_im[0].header.update(fors2_wcs_header)
fors2_wcs = WCS(fors2_wcs_header)

In [9]:
fits.writeto("W0204-0506.FORS2.ebeam.fits",np.where(emask, np.nan, fors2_im[0].data), header=fors2_im[0].header, overwrite=True)

In [10]:
#So now we need to call the ResolvedPol object just to match exactly the default stamp size used. 
bhd = ResolvedPol("W0204-0506", "R_SPECIAL", star_coords=(1086.5, 222.5), root_folder=os.getcwd()+"/../../analysis_v3")
_ = subprocess.call("rm -r work stamps", shell=True)

mkdir: work: File exists


In [11]:
#Using the wcs of each object, we'll figure out the coordinate positions of the edges and match the cutout. 
xc = 0.5*(bhd.ix1_z + bhd.ix2_z-1)
yc = 0.5*(bhd.iy1_z + bhd.iy2_z-1)
dx = bhd.ix2_z-1 - bhd.ix1_z
dy = bhd.iy2_z-1 - bhd.iy1_z
fors2_cutout = Cutout2D(fors2_im[0].data, (xc, yc), (dy, dx), wcs=fors2_wcs)
fits.writeto("fors2.cutout.fits",fors2_cutout.data, header=fors2_cutout.wcs.to_header() ,overwrite=True)

In [12]:
c_f160w_cutout = fors2_wcs.pixel_to_world(xc, yc)
dx_f160w_cutout = dx * fors2_wcs.proj_plane_pixel_scales()[0]/wcs_f160w.proj_plane_pixel_scales()[0]
dy_f160w_cutout = dy * fors2_wcs.proj_plane_pixel_scales()[1]/wcs_f160w.proj_plane_pixel_scales()[1]
dx_f160w_cutout = int(np.round(dx_f160w_cutout,0))
dy_f160w_cutout = int(np.round(dy_f160w_cutout,0))
f160w_cutout = Cutout2D(f160w['SCI'].data, c_f160w_cutout, (dy_f160w_cutout, dx_f160w_cutout), wcs=wcs_f160w)
fits.writeto("f160w.cutout.fits",f160w_cutout.data, header=f160w_cutout.wcs.to_header() ,overwrite=True)

In [13]:
wcs_f555w = WCS(f555w[0].header)
c_f555w_cutout = fors2_wcs.pixel_to_world(xc, yc)
dx_f555w_cutout = dx * fors2_wcs.proj_plane_pixel_scales()[0]/wcs_f555w.proj_plane_pixel_scales()[0]
dy_f555w_cutout = dy * fors2_wcs.proj_plane_pixel_scales()[1]/wcs_f555w.proj_plane_pixel_scales()[1]
dx_f555w_cutout = int(np.round(dx_f555w_cutout,0))
dy_f555w_cutout = int(np.round(dy_f555w_cutout,0))
f555w_cutout = Cutout2D(f555w[0].data, c_f555w_cutout, (dy_f555w_cutout, dx_f555w_cutout), wcs=wcs_f555w)
fits.writeto("f555w.cutout.fits",f555w_cutout.data, header=f555w_cutout.wcs.to_header() ,overwrite=True)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
